In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import preReq

In [2]:
df = pd.read_csv('penguins.csv')

In [3]:
df['gender'] = LabelEncoder().fit_transform(df['gender'])
scaled_df = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df.iloc[:, 1:]), columns=df.iloc[:, 1:].columns)
scaled_df['species'] = df['species']

In [4]:
selectedF = (preReq.INarr[0], preReq.INarr[1])
selectedC = (preReq.INarr[2], preReq.INarr[3])
weight = np.random.random((2))
eta = preReq.INarr[4]
epochs = preReq.INarr[5]
bias = np.random.randn() if preReq.INarr[6] else 0
MeanS= preReq.INarr[7]

---
# Visualizations

In [5]:
def visualizeData(scaled_df):  # 10 combs
    for i in range(len(preReq.Features) - 1):
        for j in range(i + 1, len(preReq.Features)):
            fL1, fL2 = preReq.Features[i], preReq.Features[j]
            f1, f2 = scaled_df[fL1], scaled_df[fL2]

            plt.xlabel(fL1)
            plt.ylabel(fL2)

            CF = [(f1[:50], f2[:50]), (f1[50:100], f2[50:100]), (f1[100:], f2[100:])]
            for plot in range(3):
                plt.scatter(CF[plot][0], CF[plot][1])

            plt.legend([preReq.Classes[0], preReq.Classes[1], preReq.Classes[2]])
            plt.show()


# visualizeData(scaled_df)

---
# Main

In [6]:
misClass = list()
for x in preReq.Classes:
    if x not in selectedC:
        misClass.append(x)

misFeatures = list()
for x in preReq.Features:
    if x not in selectedF:
        misFeatures.append(x)

In [7]:
scaled_df.drop(scaled_df.index[(scaled_df["species"] == misClass[0])], axis=0, inplace=True)
scaled_df.drop(columns=misFeatures, axis=1, inplace=True)
scaled_df['species'] = scaled_df['species'].replace(selectedC, [1, -1])

In [8]:
C1, C2 = (scaled_df[:50]).sample(frac=1), (scaled_df[50:100]).sample(frac=1)
C1_train, C1_test, C2_train, C2_test = C1[:30], C1[30:], C2[:30], C2[30:]

all_train_data = pd.concat([C1_train, C2_train]).to_numpy()
all_test_data = pd.concat([C1_test, C2_test]).to_numpy()

train_data, train_target = all_train_data[:, :2], all_train_data[:, 2:]
test_data, test_target = all_test_data[:, :2], all_test_data[:, 2:]

In [9]:
def CalculateMSE(input, actual_output, weight, bias):
    square_error = list()
    prediction = input.dot(weight.transpose()) + bias

    for i in range(len(actual_output)):
        error = np.square(actual_output[i] - prediction[i])
        square_error.append(error)
    MSE =  (1/len(square_error)) * (sum(square_error)/2)

    return MSE

In [10]:
def PerceptronAlgo(epochs, weight, bias, eta, train_data, train_target):
    import numpy as np

    for epoch in range(epochs):
        yPredTrain = (train_data.dot(weight.transpose()) + bias)
        for i in range(len(train_target)):
            if yPredTrain[i] != train_target[i]:
                loss = train_target[i] - yPredTrain[i]
                weight += (eta * loss * train_data[i])
                bias += (eta * loss) if bias else 0
        MSE = CalculateMSE(train_data,train_target,weight,bias)
        if MSE < MeanS:
            break
    return yPredTrain

yPredTrain = PerceptronAlgo(epochs, weight, bias, eta, train_data, train_target)

In [11]:
def classifyTrain():
    plt.figure("Trained Features Figure")
    plt.scatter(x=train_data[:, :1], y=train_data[:, 1:2], c=train_target)
    plt.plot(np.array([0, 1]), np.array([(-bias) / weight[1], (-weight[0] - bias) / weight[1]]))
    plt.show()


# classifyTrain()

---
# Testing

In [12]:
yPredTest = (test_data.dot(weight.transpose()) + bias)

In [13]:
def ConfuionMatrix(target, yPred):
    import numpy as np
    import pandas as pd

    testCase = list()
    confMat = np.zeros([2, 2])

    for i in range(len(yPred)):
        if target[i] == 1:
            if yPred[i] == 1:
                confMat[0][0] += 1
                testCase.append([int(target[i][0]), yPred[i], '[*]'])
            else:
                confMat[0][1] += 1
                testCase.append([int(target[i][0]), yPred[i], '[ ]'])

        elif target[i] == -1:
            if yPred[i] == -1:
                confMat[1][1] += 1
                testCase.append([int(target[i][0]), yPred[i], '[*]'])
            else:
                confMat[1][0] += 1
                testCase.append([int(target[i][0]), yPred[i], '[ ]'])

    Cases = pd.DataFrame(testCase, columns=['Real', 'Pred', 'Match'])
    return ((np.trace(confMat) / np.sum(confMat)) * 100), confMat, pd.DataFrame(Cases,
                                                                                columns=['Real', 'Pred', 'Match'])

In [14]:
preReq.OUTarr[1], confMatTrain, truthValsTrain = ConfuionMatrix(train_target, yPredTrain)
print('|> Truth Values for Training <|\n   Training Accuracy: ', preReq.OUTarr[1], '%\n', truthValsTrain, '\n',
      '-' * 35)

preReq.OUTarr[2], confMatTest, truthValsTest = ConfuionMatrix(test_target, yPredTest)
print('|> Truth Values for Testing <|\n   Testing Accuracy: ', preReq.OUTarr[2], '%\n', truthValsTest, '\n', '-' * 35)

|> Truth Values for Training <|
   Training Accuracy:  0.0 %
     Real          Pred Match
0      1  7.339160e+87   [ ]
1      1  7.559501e+87   [ ]
2      1  8.389317e+87   [ ]
3      1  7.813149e+87   [ ]
4      1  7.090080e+87   [ ]
5      1  9.119686e+87   [ ]
6      1  9.068592e+87   [ ]
7      1  7.462333e+87   [ ]
8      1  9.045328e+87   [ ]
9      1  8.487402e+87   [ ]
10     1  9.147971e+87   [ ]
11     1  1.046682e+88   [ ]
12     1  7.025755e+87   [ ]
13     1  9.074067e+87   [ ]
14     1  8.719148e+87   [ ]
15     1  7.525286e+87   [ ]
16     1  8.720973e+87   [ ]
17     1  8.629735e+87   [ ]
18     1  8.229196e+87   [ ]
19     1  8.719148e+87   [ ]
20     1  8.960473e+87   [ ]
21     1  9.142496e+87   [ ]
22     1  7.418539e+87   [ ]
23     1  8.242426e+87   [ ]
24     1  7.574556e+87   [ ]
25     1  7.946356e+87   [ ]
26     1  8.156663e+87   [ ]
27     1  9.119686e+87   [ ]
28     1  9.460922e+87   [ ]
29     1  8.092339e+87   [ ]
30    -1  7.524823e+87   [ ]
31    -1  

In [15]:
preReq.OUTarr[0] = selectedC[0] if sum(yPredTest) else selectedC[1]
preReq.OUTarr[3][0] = confMatTest[0][0]
preReq.OUTarr[3][1] = confMatTest[0][1]
preReq.OUTarr[3][2] = confMatTest[1][0]
preReq.OUTarr[3][3] = confMatTest[1][1]